## Performance evaluation: LLM-assisted QnA

**Overview**: In this example, we use UpTrain to evaluate the performance of a chat application used for natural language search against technical documenation. 

**Why is monitoring needed**: LLMs used for retrieval can often produce unconstrained output. Even with custom prompts designed to coerce the output, the response can be irrelevant to the query. Further, for technical documentation, misleading responses can end up wasting more time than no response at all.

Monitoring NLP tasks with traditional metrics (such as accuracy) in production is hard, as groud truth is unavailable (or extremely delayed when there is a human in the loop).

**Problem**: The workflow of our hypothetical chat application goes like,
- User enters a natural language query. 
- The query is converted to an embedding, and relevant sections from the documentation are retrieved using nearest neighbor search. 
- The original query along with the retrieved sections are passed to a language model (LM), along with a custom prompt to generate a response. 

We use a dataset built from logs generated by a chatbot made to answer questions from the [Streamlit user documentation](https://docs.streamlit.io/). 

**Solution**: We illustate how to use the Uptrain Evals framework to assess the performance of the chatbot. 

## Install required packages

```bash
pip install uptrain[full] # Install UpTrain with all dependencies
```

In [1]:
import os
import polars as pl

SCRATCH_SPACE = "/tmp/uptrain-scratch/"
os.makedirs(SCRATCH_SPACE, exist_ok=True)

Uptrain primitives share configuration like openai api keys, or the logs folder to write to through a single `Settings` object. 

In [2]:
from uptrain.framework import Settings

UPTRAIN_LOGS_DIR = os.path.join(SCRATCH_SPACE, "logs")
UPTRAIN_SETTINGS = Settings(logs_folder=UPTRAIN_LOGS_DIR)

In [3]:
# download the dataset if not present
url = "https://oodles-dev-training-data.s3.us-west-1.amazonaws.com/qna-streamlit-docs.jsonl"
dataset_path = os.path.join(SCRATCH_SPACE, "qna-notebook-data.jsonl")

if not os.path.exists(dataset_path):
    import httpx
    r = httpx.get(url)
    with open(dataset_path, "wb") as f:
        f.write(r.content)

#### explore the dataset

In [4]:
dataset = pl.read_ndjson(dataset_path)

This dataset has multiple log entries corresponding to a single user query. 

### Evaluations

We can do a few different checks to see if the chatbot is working as expected. 

**Response Document similarity**

We want to make sure the LLM responses are close to the document provided as context and it is not hallucinating technical terms. A quick proxy for this is to compare how similar the embeddings of the response and the document are.

Uptrain operators are implemented as regular dataset transformations that take a polars dataframe as input, and produce another. This makes it easy to integrate them into existing workflows. 

In [11]:
from uptrain.operators import CosineSimilarity, RougeScore
from uptrain.framework import Check

op_1 =         RougeScore(
            score_type="precision",
            col_in_generated="response", 
            col_in_source="document_text",
            col_out="hallucination-score",
        )

op_2 = CosineSimilarity(
            col_in_vector_1="question_embeddings",
            col_in_vector_2="context_embeddings",
            col_out="similarity-question-context",
        )

out_df = dataset
for op in [op_1, op_2]:
    out_df = op.setup(UPTRAIN_SETTINGS).run(out_df)["output"]

We could visualize the distribution of hallucination scores by doing a quick histogram plot.

In [12]:
import plotly.express as px

px.histogram(out_df.to_pandas(), x="hallucination-score")

In [8]:
import plotly.express as px

px.histogram(out_df.to_pandas(), x="similarity-question-context")

**How similar are the documents retrieved for a query?**

We could assess how good our ANN based retrieval is working by looking at the distribution of the cosine similarity between all the retrieved documents. 

TODO: Does a dispersion mean better retrieval?

In [9]:
from uptrain.operators import Distribution

op_2 = Distribution(
    kind="rouge",
    col_in_embs=["document_text"],
    col_in_groupby=["question_idx", "experiment_id"],
    col_out=["rogue_f1"],
)

out_df = op_2.setup(UPTRAIN_SETTINGS).run(dataset)["output"]

In [10]:
px.histogram(out_df.to_pandas(), x="rogue_f1", nbins=20)

### Running multiple evaluations together

While you could use uptrain to run evaluations one at a time, it can often be useful to include it as part of your automated testing workflow. Uptrain provides the concept of a `Check set` to make this easier - you list a set of checks you want to run, uptrain runs them all together at the defined schedule and provides you a dashboard to analyze the results. 

In [13]:
from uptrain.framework import CheckSet, Check
from uptrain.operators.io import JsonReader
from uptrain.operators import PlotlyChart

list_checks = [
    Check(name="check_1", operators=[op_1], plot=[
        PlotlyChart.Histogram(x="hallucination-score", nbins=20),
        PlotlyChart.Histogram(x="similarity-question-context", nbins=20),
    ]),
    Check(name="check_2", operators=[op_2], plot=[
        PlotlyChart.Histogram(x="rogue_f1", nbins=20),
    ])    
]

check_set = CheckSet(
    source=JsonReader(fpath=dataset_path),
    checks=list_checks
)

In [14]:
settings=Settings(logs_folder=UPTRAIN_LOGS_DIR)
check_set.setup(settings).run()

2023-07-04 11:59:40.541 | DEBUG    | uptrain.framework.base:run:119 - Executing node: sequence_0 for operator DAG: check_1
2023-07-04 11:59:41.642 | DEBUG    | uptrain.framework.base:run:119 - Executing node: sequence_0 for operator DAG: check_2


In [ ]:
# once run, you can start up a streamlit dashboard against the logs folder to see the results

from uptrain.dashboard import StreamlitRunner

runner = StreamlitRunner(LOGS_DIR)
runner.start()